In [1]:
%pip install --quiet transformers==4.46.1 "unstructured[pdf,docx]==0.10.30" llama-index==0.9.3 pydantic==2.9.2 mlflow==2.12.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 886.7/886.7 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 46.5 MB/s eta 0:00

In [2]:
!pip install pyspark

In [14]:
import nltk
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger_eng')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


In [3]:
# Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Check Spark Session Information
spark


In [4]:
# Reduce the arrow batch size as our PDF can be big in memory
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", 10)

In [5]:
articles_path = f"pdfs/"
#table_name = f"{DA.catalog_name}.{DA.schema_name}.pdf_raw_text"

# read pdf files
df = (
        spark.read.format("binaryfile")
        .option("recursiveFileLookup", "true")
        .load(articles_path)
        )

# save list of the files to table
df.write.mode("overwrite").coalesce(1).saveAsTable('nvidia')
display(df)

DataFrame[path: string, modificationTime: timestamp, length: bigint, content: binary]

In [6]:
df.show()

+--------------------+--------------------+------+--------------------+
|                path|    modificationTime|length|             content|
+--------------------+--------------------+------+--------------------+
|file:/content/pdf...|2024-11-13 06:22:...| 94358|[25 50 44 46 2D 3...|
|file:/content/pdf...|2024-11-13 06:22:...| 93752|[25 50 44 46 2D 3...|
|file:/content/pdf...|2024-11-13 06:22:...| 92940|[25 50 44 46 2D 3...|
|file:/content/pdf...|2024-11-13 06:22:...| 89308|[25 50 44 46 2D 3...|
|file:/content/pdf...|2024-11-13 06:22:...| 87619|[25 50 44 46 2D 3...|
|file:/content/pdf...|2024-11-13 06:22:...| 84427|[25 50 44 46 2D 3...|
|file:/content/pdf...|2024-11-13 06:22:...| 81889|[25 50 44 46 2D 3...|
|file:/content/pdf...|2024-11-13 06:22:...| 80686|[25 50 44 46 2D 3...|
|file:/content/pdf...|2024-11-13 06:22:...| 80629|[25 50 44 46 2D 3...|
|file:/content/pdf...|2024-11-13 06:22:...| 80248|[25 50 44 46 2D 3...|
|file:/content/pdf...|2024-11-13 06:22:...| 76255|[25 50 44 46 2

In [7]:
import time
from unstructured.partition.auto import partition
import re
import io

def extract_doc_text(x : bytes) -> str:
  # Read files and extract the values with unstructured
  sections = partition(file=io.BytesIO(x))
  def clean_section(txt):
    txt = re.sub(r'\n', '', txt)
    return re.sub(r' ?\.', '.', txt)
  # Default split is by section of document
  # concatenate them all together because we want to split by sentence instead.
  return "\n".join([clean_section(s.text) for s in sections])


In [15]:
with open(f"pdfs/Filing Date - 2022-Aug-30 - NVDA.OQ - NVIDIA Corp - 10-Q - NVIDIA CORP 10-Q - 221218592.pdf", mode="rb") as pdf:
  doc = extract_doc_text(pdf.read())
  print(doc)

>
ITEM 2. MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS Forward-Looking Statements This Quarterly Report on Form 10-Q contains forward-looking statements which are based on our management’s beliefs and assumptions and on information currently available to our management. In some cases, you can identify forward-looking statements by terms such as “may,” “will,” “should,” “could,” “goal,” “would,” “expect,” “plan,” “anticipate,” “believe,” “estimate,” “project,” “predict,” “potential” and similar expressions intended to identify forward-looking statements. These statements involve known and unknown risks, uncertainties and other factors, which may cause our actual results, performance, time frames or achievements to be materially different from any future results, performance, time frames or achievements expressed or implied by the forward-looking statements. We discuss many of these risks, uncertainties and other factors in this Quarterly Report o

In [16]:
import io
import os
import pandas as pd

from llama_index.langchain_helpers.text_splitter import SentenceSplitter
from llama_index import Document, set_global_tokenizer
from transformers import AutoTokenizer
from typing import Iterator
from pyspark.sql.functions import col, udf, length, pandas_udf, explode
from unstructured.partition.auto import partition


@pandas_udf("array<string>")
def read_as_chunk(batch_iter: Iterator[pd.Series]) -> Iterator[pd.Series]:
    #set llama2 as tokenizer
    set_global_tokenizer(
      AutoTokenizer.from_pretrained("hf-internal-testing/llama-tokenizer")
    )
    #Sentence splitter from llama_index to split on sentences
    splitter = SentenceSplitter(chunk_size=500, chunk_overlap=50)
    def extract_and_split(b):
      txt = extract_doc_text(b)
      nodes = splitter.get_nodes_from_documents([Document(text=txt)])
      return [n.text for n in nodes]

    for x in batch_iter:
        yield x.apply(extract_and_split)

In [17]:
df_chunks = (df
                .withColumn("content", explode(read_as_chunk("content")))
                .selectExpr('path as pdf_name', 'content')
                )
display(df_chunks)

DataFrame[pdf_name: string, content: string]

In [18]:
df_chunks.show()

+--------------------+--------------------+
|            pdf_name|             content|
+--------------------+--------------------+
|file:/content/pdf...|="i6311855089984a...|
|file:/content/pdf...|All references to...|
|file:/content/pdf...|Hopper architectu...|
|file:/content/pdf...|The computing ind...|
|file:/content/pdf...|In July 2023, the...|
|file:/content/pdf...|As a percentage o...|
|file:/content/pdf...|Our product and s...|
|file:/content/pdf...|We have not exper...|
|file:/content/pdf...|Data Center reven...|
|file:/content/pdf...|We had solid dema...|
|file:/content/pdf...|We introduced an ...|
|file:/content/pdf...|Financial Informa...|
|file:/content/pdf...|4 (0.5) 0.4 1.3 5...|
|file:/content/pdf...|084 $\n14,862 $ 5...|
|file:/content/pdf...|Revenue from GPU ...|
|file:/content/pdf...|We refer to custo...|
|file:/content/pdf...|Gross Profit and ...|
|file:/content/pdf...|The net effect on...|
|file:/content/pdf...|For fiscal year 2...|
|file:/content/pdf...|For fiscal

In [19]:
df_chunks.write.format('csv').save('chunks')